In [ ]:
# Can be used to save files on Gdrive
# from google.colab import drive
# drive.mount('/content/gdrive')
# path = "/content/gdrive/My Drive/Colab Notebooks"

# Load the TensorBoard notebook extension
# %load_ext tensorboard
# from datetime import datetime
# from packaging import version

# print("TensorFlow version: ", tf.__version__)

In [ ]:
import numpy as np
import math
import tensorflow as tf
import tensorflow.keras as keras
import plotly.graph_objects as go

In [ ]:
class print_dots(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if epoch % 1000 == 0: print('')
        if epoch % 10 == 0: print('.', end='')
    def on_train_end(self, epoch, logs=None):
        print('\nTraining is complete')

def print_training_information(history):
    mae_metric = history.history['mae']
    val_mae = history.history['val_mae']
    print('mae training {0:g}; mae validation {1:g}'.format(mae_metric[-1], val_mae[-1]))

def plot_history(fig, history):
    loss = np.sqrt( history.history['loss'] )
    metric = np.sqrt( history.history['mse'] )
    val = np.sqrt( history.history['val_mse'] )
    fig.add_trace(go.Scatter(y=loss,name='loss'))
    fig.add_trace(go.Scatter(y=metric,name='error training'))
    fig.add_trace(go.Scatter(y=val, name='error validation'))
    fig.update_layout(height=400, width=600, yaxis_type="log", xaxis_title='epoch')

def lr_exponential_decay(epoch, lr):
  growth_rate = final_learning_rate / initial_learning_rate
  lr_ = initial_learning_rate * growth_rate**(epoch / (n_epochs-1.))
  tf.summary.scalar("learning rate", data=lr_, step=epoch)
  metric_lr[epoch] = lr_  
  return lr_

def lr_constant(epoch, lr):
  tf.summary.scalar("learning rate", data=learning_rate, step=epoch)
  metric_lr[epoch] = learning_rate
  return learning_rate

In [ ]:
optimr = keras.optimizers.SGD()

In [ ]:
dnn_lr = keras.models.Sequential()
dnn_lr.add(keras.layers.InputLayer(input_shape=1))
dnn_lr.add(keras.layers.Dense(8, activation='tanh'))
dnn_lr.add(keras.layers.Dense(1, activation="linear"))

dnn_lr.compile(loss='mse', optimizer=optimr, metrics=['mse','mae'])

x_t = [-1, 0, 1]
y_t = [-1, 0, 1]

x_t = np.array(x_t, dtype = np.float32)
y_t = np.array(y_t, dtype = np.float32)

x_v = np.linspace(-1.,1.,4)
y_v = x_v

n_epochs = 10
metric_lr = np.zeros((n_epochs,))
initial_learning_rate = 1e-1
final_learning_rate = 1

# lr_callback = keras.callbacks.LearningRateScheduler(lr_constant)
lr_callback = keras.callbacks.LearningRateScheduler(lr_exponential_decay)

history = dnn_lr.fit(x_t, y_t,           
                  validation_data=(x_v,y_v),         
                  epochs=n_epochs,
                  batch_size=int(x_t.size),
                  callbacks = [lr_callback,print_dots()], 
                  verbose = 0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_t, y=y_t, mode='markers', name='training'))
x_p = np.linspace(-1,1,8)
fig.add_trace(go.Scatter(x=x_p,                         
                         y=np.array(dnn_lr.predict(x_p) ).reshape((x_p.size,)),
                         name='prediction'))
fig.update_layout(height=400, width=600)
fig.show()

fig = go.Figure()
plot_history(fig, history)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(y=metric_lr))
fig.update_layout(height=400, width=550, yaxis_type='log', 
                  xaxis_title='epoch', yaxis_title='learning rate')
fig.show()

In [ ]:
dnn_val = keras.models.Sequential()
dnn_val.add(keras.layers.InputLayer(input_shape=1))
dnn_val.add(keras.layers.Dense(8, activation='tanh'))
dnn_val.add(keras.layers.Dense(1, activation="linear"))

In [ ]:
n_epochs = 100
metric_lr = np.zeros((n_epochs,))
initial_learning_rate = 0.2
final_learning_rate = 0.2
dnn_val.compile(loss='mse', optimizer=optimr, metrics=['mse','mae'])

x_t = [-1, -0.1, 0.1, 1]
y_t = [-1, 0, 0, 1]

x_t = np.array(x_t, dtype = np.float32)
y_t = np.array(y_t, dtype = np.float32)

x_v = np.linspace(-1.,1.,8)
y_v = x_v

history = dnn_val.fit(x_t, y_t,           
                  validation_data=(x_v,y_v),         
                  epochs=n_epochs,
                  batch_size=int(x_t.size),
                  callbacks = [lr_callback,print_dots()], 
                  verbose = 0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_t, y=y_t, mode='markers', name='training'))
x_p = np.linspace(-1,1,32)
fig.add_trace(go.Scatter(x=x_p, mode='lines',                        
                         y=np.array(dnn_val.predict(x_p)).reshape((x_p.size,)),
                         name='prediction'))
fig.update_layout(height=400, width=600)
fig.show()

fig = go.Figure()
plot_history(fig, history)
fig.show()

In [ ]:
dnn_val = keras.models.Sequential()
dnn_val.add(keras.layers.InputLayer(input_shape=1))
dnn_val.add(keras.layers.Dense(8, activation='tanh',
            kernel_initializer=tf.random_uniform_initializer(minval=-1.5, maxval=1.5)
            ))
dnn_val.add(keras.layers.Dense(1, activation="linear",
            kernel_initializer=tf.random_uniform_initializer(minval=-1.5, maxval=1.5)
            ))

n_epochs = 100
metric_lr = np.zeros((n_epochs,))
initial_learning_rate = 0.1
final_learning_rate = 0.1
dnn_val.compile(loss='mse', optimizer=optimr, metrics=['mse','mae'])

history = dnn_val.fit(x_t, y_t,           
                  validation_data=(x_v,y_v),         
                  epochs=n_epochs,
                  batch_size=int(x_t.size),
                  callbacks = [lr_callback,print_dots()], 
                  verbose = 0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_t, y=y_t, mode='markers', name='training'))
x_p = np.linspace(-1,1,32)
fig.add_trace(go.Scatter(x=x_p, mode='lines',                        
                         y=np.array(dnn_val.predict(x_p)).reshape((x_p.size,)),
                         name='prediction'))
fig.update_layout(height=400, width=600)
fig.show()

fig = go.Figure()
plot_history(fig, history)
fig.show()

In [ ]:
x = np.linspace(-3,3,64)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.tanh(x)))
fig.update_layout(height=400, width=600, xaxis_title='x',yaxis_title='tanh')
fig.show()

In [ ]:
dnn_val = keras.models.Sequential()
dnn_val.add(keras.layers.InputLayer(input_shape=1))
dnn_val.add(keras.layers.Dense(8, activation='tanh',
            kernel_initializer=tf.random_uniform_initializer(minval=-10, maxval=10)
            ))
dnn_val.add(keras.layers.Dense(1, activation="linear",
            kernel_initializer=tf.random_uniform_initializer(minval=-10, maxval=10)
            ))

n_epochs = 100
metric_lr = np.zeros((n_epochs,))
initial_learning_rate = 0.1
final_learning_rate = 0.1
dnn_val.compile(loss='mse', optimizer=optimr, metrics=['mse','mae'])

history = dnn_val.fit(x_t, y_t,           
                  validation_data=(x_v,y_v),         
                  epochs=n_epochs,
                  batch_size=int(x_t.size),
                  callbacks = [lr_callback,print_dots()], 
                  verbose = 0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_t, y=y_t, mode='markers', name='training'))
x_p = np.linspace(-1,1,256)
fig.add_trace(go.Scatter(x=x_p, mode='lines',                        
                         y=np.array(dnn_val.predict(x_p)).reshape((x_p.size,)),
                         name='prediction'))
fig.update_layout(height=400, width=600)
fig.show()

fig = go.Figure()
plot_history(fig, history)
fig.show()

In [ ]:
small = keras.models.Sequential()
small.add(keras.layers.InputLayer(input_shape=1))
small.add(keras.layers.Dense(1, activation='tanh'))
small.add(keras.layers.Dense(1, activation="linear"))

n_epochs = 100
metric_lr = np.zeros((n_epochs,))
initial_learning_rate = 0.1
final_learning_rate = 0.1
small.compile(loss='mse', optimizer=optimr, metrics=['mse','mae'])

history = small.fit(x_t, y_t,           
                  validation_data=(x_v,y_v),         
                  epochs=n_epochs,
                  batch_size=int(x_t.size),
                  callbacks = [lr_callback,print_dots()], 
                  verbose = 0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_t, y=y_t, mode='markers', name='training'))
x_p = np.linspace(-1,1,256)
fig.add_trace(go.Scatter(x=x_p, mode='lines',                        
                         y=np.array(small.predict(x_p)).reshape((x_p.size,)),
                         name='prediction'))
fig.update_layout(height=400, width=600)
fig.show()

fig = go.Figure()
plot_history(fig, history)
fig.show()

In [ ]:
l2reg = 1e-2

regd = keras.models.Sequential()
regd.add(keras.layers.InputLayer(input_shape=1))
regd.add(keras.layers.Dense(8, activation='tanh',
                          kernel_regularizer=tf.keras.regularizers.l2(l=l2reg),
                          bias_regularizer=tf.keras.regularizers.l2(l=l2reg)))
regd.add(keras.layers.Dense(1, activation="linear"))

n_epochs = 100
metric_lr = np.zeros((n_epochs,))
initial_learning_rate = 0.1
final_learning_rate = 0.1
regd.compile(loss='mse', optimizer=optimr, metrics=['mse','mae'])

history = regd.fit(x_t, y_t,           
                  validation_data=(x_v,y_v),         
                  epochs=n_epochs,
                  batch_size=int(x_t.size),
                  callbacks = [lr_callback,print_dots()], 
                  verbose = 0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_t, y=y_t, mode='markers', name='training'))
x_p = np.linspace(-1,1,256)
fig.add_trace(go.Scatter(x=x_p, mode='lines',                        
                         y=np.array(regd.predict(x_p)).reshape((x_p.size,)),
                         name='prediction'))
fig.update_layout(height=400, width=600)
fig.show()

fig = go.Figure()
plot_history(fig, history)
fig.show()